### 專案名稱: 機器學習練習
### 功能描述: 讀取新北市托育中心資料
### https://data.ntpc.gov.tw/api/v1/rest/datastore/382000000A-000139-001
### 版權所有: Dunk  
### 程式撰寫: Dunk  
### 撰寫日期：2020/04/13
### 改版日期:  
### 改版備註:

### 新北市私立托嬰機構名冊
[說明網頁](https://data.gov.tw/dataset/26869 "說明網頁")

- 資料集描述: 
>- 新北市政府社會局所提供的托嬰機構名冊主要提供欄位包含名稱、地址、電話、可收托人數、備註

- 主要欄位說明: 
>- title(名稱)
>- address(地址)
>- tel(電話)
>- avaliable(可收托人數)
>- remark(備註)
>- twd97X(twd97緯度)
>- twd97Y(twd97經度)
>- wgs84aX(wgs84a緯度)
>- wgs84aY(wgs84a經度)

設計出區類別，並採用獨熱編碼 + 線性迴歸(由計算結果)
- 非深度學習時，類別型特徵建議預設採標籤編碼;
- 深度學習時，預設採獨熱編碼

#### 呼叫所需套件

In [30]:
# 讀取json資料
import json
import requests
import pprint
import pandas as pd
import time
#特徵工程
from sklearn.preprocessing import MinMaxScaler,LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor

#### 資料前置處理

In [31]:
url = r'https://data.ntpc.gov.tw/api/v1/rest/datastore/382000000A-000139-001'
df_babysite = pd.read_json(url)
#資料都在Result列內
df_baby_data = pd.DataFrame(df_babysite.loc['records','result'])
# remark欄位檢視，發現沒有值，直接移除
df_baby_data = df_baby_data.drop(['remark'],axis =1)
# 增加區分類
df_baby_data['區'] = df_baby_data['address'].apply(lambda x: x.split('區')[0]+'區')

train_Y = df_baby_data['avaliable']
df = df_baby_data.drop(['avaliable'] , axis=1)
df

,title,address,tel,twd97X,twd97Y,wgs84aX,wgs84aY,區
0,新北市私立寶貝媽咪托嬰中心,新北市板橋區莊敬路46號2樓,82529955,297696.69,2768902.08,121.472626809278,25.0274579217759,新北市板橋區
1,新北市私立卡爾威特托嬰中心,新北市板橋區板新路101號1樓、107號2樓,89518905,297713.66,2767424.19,121.472743870936,25.0141149285452,新北市板橋區
2,新北市私立僾家寶托嬰中心,新北市板橋區裕民街122號2樓,82595600,296340.08,2768910.73,121.459184790012,25.0275781110774,新北市板橋區
3,新北市私立捧馨園托嬰中心,新北市板橋區四川路1段151號,29563008,296433.28,2766327.39,121.460021425451,25.0042527242484,新北市板橋區
4,新北市私立喜閱寶寶托嬰中心,新北市板橋區廣和街61號1.2樓,89516588,296698.15,2765085.42,121.462603499667,24.9930318926334,新北市板橋區
...,...,...,...,...,...,...,...,...
156,新北市私立悅萌園托嬰中心,新北市林口區文化一路二段173號,26012838,288638.53,2774864.56,121.383038442314,25.0815461198244,新北市林口區
157,新北市私立佳甯蒙特梭利托嬰中心,新北市林口區三民路122號1樓2樓,26093162,286396.17,2773666.62,121.360777774151,25.0707866905547,新北市林口區
158,新北市私立長青托嬰中心,新北市鶯歌區鶯桃路188號2樓,26794011,284106.24,2761247,121.337772223512,24.9587132410024,新北市鶯歌區
159,新北市私立欣欣寶貝屋托嬰中心,新北市鶯歌區德昌二街49號1-3樓,26783958,283666.15,2761226.06,121.333413336467,24.9585339907318,新北市鶯歌區


In [32]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Object Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

8 Object Features : ['title', 'address', 'tel', 'twd97X', 'twd97Y', 'wgs84aX', 'wgs84aY', '區']



,title,address,tel,twd97X,twd97Y,wgs84aX,wgs84aY,區
0,新北市私立寶貝媽咪托嬰中心,新北市板橋區莊敬路46號2樓,82529955,297696.69,2768902.08,121.472626809278,25.0274579217759,新北市板橋區
1,新北市私立卡爾威特托嬰中心,新北市板橋區板新路101號1樓、107號2樓,89518905,297713.66,2767424.19,121.472743870936,25.0141149285452,新北市板橋區
2,新北市私立僾家寶托嬰中心,新北市板橋區裕民街122號2樓,82595600,296340.08,2768910.73,121.459184790012,25.0275781110774,新北市板橋區
3,新北市私立捧馨園托嬰中心,新北市板橋區四川路1段151號,29563008,296433.28,2766327.39,121.460021425451,25.0042527242484,新北市板橋區
4,新北市私立喜閱寶寶托嬰中心,新北市板橋區廣和街61號1.2樓,89516588,296698.15,2765085.42,121.462603499667,24.9930318926334,新北市板橋區


In [37]:
# 獨熱編碼 + 線性迴歸
df_temp = pd.get_dummies(df)
train_X = df_temp[:train_num]
estimator = LinearRegression()
start = time.time()
print(f'shape : {train_X.shape}')
print(f'score : {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (161, 1138)
score : -0.07664086953945759
time : 0.4240000247955322 sec
